In [2]:
import struct
import numpy as np
from array import array
import matplotlib.pyplot as plt
import numpy.linalg as LA
import pandas as pd
from pandas import ExcelWriter, ExcelFile, read_excel, DataFrame
from openpyxl import load_workbook
from itertools import product

In [3]:
def readExcelSheet1(excelfile):
    return (read_excel(excelfile)).values

In [4]:
def readExcelRange(excelfile,sheetname="Sheet1",startrow=1,endrow=1,startcol=1,endcol=1):
    values=(read_excel(excelfile, sheetname,header=None)).values;
    return values[startrow-1:endrow,startcol-1:endcol]

In [5]:
def readExcel(excelfile,**args):
    if args:
        data=readExcelRange(excelfile,**args)
    else:
        data=readExcelSheet1(excelfile)
    [nr,nc]=data.shape
    if [nr,nc]==[1,1]:
        return data[0,0]
    elif min([nr,nc])==1:
        return np.squeeze(data)
    else:
        return data

In [6]:
def writeExcelData(x,excelfile,sheetname,startrow,startcol):
    df=DataFrame(x)
    book = load_workbook(excelfile)
    writer = ExcelWriter(excelfile, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, sheet_name=sheetname,startrow=startrow-1, startcol=startcol-1, header=False, index=False)
    writer.save()
    writer.close()

In [7]:
def getSheetNames(excelfile):
    return (ExcelFile(excelfile)).sheet_names

In [8]:
excelfile_train = r'/Users/heart/Desktop/learning/Py&ML/project/fashion-mnist_train.csv'
excelfile_test = r'/Users/heart/Desktop/learning/Py&ML/project/fashion-mnist_test.csv'

In [9]:
Xtrain = pd.read_csv(excelfile_train).values
Xtest = pd.read_csv(excelfile_test).values

Ytrain = Xtrain[:,0]
Ytest = Xtest[:,0]
Xtrain = Xtrain[:,1:]
Xtest = Xtest[:,1:]


## *Histogram and Bayesian using PCA*

In [10]:
np.unique(Xtrain[:,0])

array([ 0,  1,  2,  3,  4,  7, 14, 16])

In [11]:
μ = np.mean(Xtrain,axis=0)
Z = Xtrain - μ
C = np.dot(Z.T,Z)/(len(μ)-1)
[λ,V] = LA.eigh(C)
λ = np.flipud(λ)
V = np.flipud(V.T)
P = np.dot(Z,V.T)

v1 = V[0,:]
v2 = V[1,:]
PC = P[:,0:2]
RP_aprox = np.dot(PC,V[0:2,:])
X_aprox = RP_aprox + μ

In [12]:
(λ[0]+λ[1])/sum(λ)

0.4673902181151126

In [43]:
# PC: contain the PC1 & PC2

range_PC1 = [np.min(PC[:,0]), np.max(PC[:,0])]
range_PC2 = [np.min(PC[:,1]), np.max(PC[:,1])]

B = 32

def Build_2DHistogramClassifier(B,PC,Ytrain,Xtrain,range_PC1,range_PC2):
    Hlabel = np.zeros([10,B,B]).astype('int32')
    PC1_bins = np.round((B-1)*(PC[:,0].astype(float)-range_PC1[0])/(range_PC1[1]-range_PC1[0])).astype('int32')
    PC2_bins = np.round((B-1)*(PC[:,1].astype(float)-range_PC2[0])/(range_PC2[1]-range_PC2[0])).astype('int32')
    for i in range(Xtrain.shape[0]):
        Hlabel[Ytrain[i],PC1_bins[i],PC2_bins[i]] += 1
    return Hlabel
Hlabel = Build_2DHistogramClassifier(B,PC,Ytrain,Xtrain,range_PC1,range_PC2)

###############

Mulabel = np.array([np.mean(PC[np.argwhere(Ytrain == x)[:,0],:],axis=0) for x in np.unique(Ytrain)])
Covlabel = np.array([np.cov(PC[np.argwhere(Ytrain == x)[:,0],:].T.astype(float)) for x in np.unique(Ytrain)])

def Gauss_2d_pdf(x,mu,sigma):
    d = np.shape(sigma)[0]
    exp_item = np.exp(-np.dot(np.dot(np.asarray(x-mu),np.linalg.inv(sigma)),np.asarray(x-mu).T)/2)
    return exp_item /((2*np.pi)**(d/2) *np.sqrt(np.abs(np.linalg.det(sigma))))


In [46]:
Ztrain = Xtrain- μ
Ptrain = np.dot(Ztrain,np.array([v1,v2]).T)
Rtrain = np.dot(Ptrain,np.array([v1,v2]))
Xtrainrec = Rtrain + μ


pc1_trainbin_loc = np.round((B-1)*(Ptrain[:,0].astype(float)-range_PC1[0])/(range_PC1[1]-range_PC1[0])).astype('int32')
pc2_trainbin_loc = np.round((B-1)*(Ptrain[:,1].astype(float)-range_PC2[0])/(range_PC2[1]-range_PC2[0])).astype('int32')

Histogram_Label = np.zeros([10,Xtrain.shape[0]]).astype('int32')
Histogram_denominator = np.zeros(Xtrain.shape[0]).astype('int32')
for i in range(10):
    Histogram_Label[i,:] = Hlabel[i,pc1_trainbin_loc,pc2_trainbin_loc]
    Histogram_denominator += Histogram_Label[i,:]
    
Histogram_classifier = np.zeros([10,Xtrain.shape[0]])
for i in range(10):
    Histogram_classifier[i,:] = Histogram_Label[i,:]/Histogram_denominator
    Histogram_classifier[i,np.argwhere(np.isnan(Histogram_classifier[i,:]))] = 0

Digits = [x for x in range(10)]
DH = np.zeros(Xtrain.shape[0]).astype('int32')
PH = np.zeros(Xtrain.shape[0])
for j in range(Xtrain.shape[0]):
    DH[j] = Digits[np.argmax(Histogram_classifier[:,j])]
    PH[j] = np.max(Histogram_classifier[:,j])
    


Bayesian_Label = np.zeros([10,Xtrain.shape[0]])
Bayesian_denominator = np.zeros(Xtrain.shape[0])
for i in range(10):
    Bayesian_Label[i,:] = np.array([Gauss_2d_pdf(x,Mulabel[i],Covlabel[i]) for x in Ptrain])
    Bayesian_denominator += Bayesian_Label[i,:]
    
Bayesian_classifier = np.zeros([10,Xtrain.shape[0]])
for i in range(10):
    Bayesian_classifier[i,:] = Bayesian_Label[i,:]/Bayesian_denominator
    Bayesian_classifier[i,np.argwhere(np.isnan(Bayesian_classifier[i,:]))] = 0

Digits = [x for x in range(10)]
DB = np.zeros(Xtrain.shape[0]).astype('int32')
PB = np.zeros(Xtrain.shape[0])
for j in range(Xtrain.shape[0]):
    DB[j] = Digits[np.argmax(Bayesian_classifier[:,j])]
    PB[j] = np.max(Bayesian_classifier[:,j])

TrainAccurancy_H = 0.
TrainAccurancy_B = 0.
for i in range(Xtrain.shape[0]):
    TrainAccurancy_H += np.sum(Ytrain[i] == DH[i])/Xtrain.shape[0]
    TrainAccurancy_B += np.sum(Ytrain[i] == DB[i])/Xtrain.shape[0]




In [47]:
print(TrainAccurancy_H)
print(TrainAccurancy_B)

0.5669833333332758
0.5226833333332807


In [41]:
Ztest = Xtest - μ
Ptest = np.dot(Ztest,np.array([v1,v2]).T)
Rtest = np.dot(Ptest,np.array([v1,v2]))
Xtestrec = Rtest + μ


pc1_testbin_loc = np.round((B-1)*(Ptest[:,0].astype(float)-range_PC1[0])/(range_PC1[1]-range_PC1[0])).astype('int32')
pc2_testbin_loc = np.round((B-1)*(Ptest[:,1].astype(float)-range_PC2[0])/(range_PC2[1]-range_PC2[0])).astype('int32')

Histogram_Label = np.zeros([10,Xtest.shape[0]]).astype('int32')
Histogram_denominator = np.zeros(Xtest.shape[0]).astype('int32')
for i in range(10):
    Histogram_Label[i,:] = Hlabel[i,pc1_testbin_loc,pc2_testbin_loc]
    Histogram_denominator += Histogram_Label[i,:]
    
Histogram_classifier = np.zeros([10,Xtest.shape[0]])
for i in range(10):
    Histogram_classifier[i,:] = Histogram_Label[i,:]/Histogram_denominator
    Histogram_classifier[i,np.argwhere(np.isnan(Histogram_classifier[i,:]))] = 0

Digits = [x for x in range(10)]
DH = np.zeros(Xtest.shape[0]).astype('int32')
PH = np.zeros(Xtest.shape[0])
for j in range(Xtest.shape[0]):
    DH[j] = Digits[np.argmax(Histogram_classifier[:,j])]
    PH[j] = np.max(Histogram_classifier[:,j])
    


Bayesian_Label = np.zeros([10,Xtest.shape[0]])
Bayesian_denominator = np.zeros(Xtest.shape[0])
for i in range(10):
    Bayesian_Label[i,:] = np.array([Gauss_2d_pdf(x,Mulabel[i],Covlabel[i]) for x in Ptest])
    Bayesian_denominator += Bayesian_Label[i,:]
    
Bayesian_classifier = np.zeros([10,Xtest.shape[0]])
for i in range(10):
    Bayesian_classifier[i,:] = Bayesian_Label[i,:]/Bayesian_denominator
    Bayesian_classifier[i,np.argwhere(np.isnan(Bayesian_classifier[i,:]))] = 0

Digits = [x for x in range(10)]
DB = np.zeros(Xtest.shape[0]).astype('int32')
PB = np.zeros(Xtest.shape[0])
for j in range(Xtest.shape[0]):
    DB[j] = Digits[np.argmax(Bayesian_classifier[:,j])]
    PB[j] = np.max(Bayesian_classifier[:,j])

TestAccurancy_H = 0.
TestAccurancy_B = 0.
for i in range(Xtest.shape[0]):
    TestAccurancy_H += np.sum(Ytest[i] == DH[i])/Xtest.shape[0]
    TestAccurancy_B += np.sum(Ytest[i] == DB[i])/Xtest.shape[0]



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


In [42]:
print(TestAccurancy_H)
print(TestAccurancy_B)

0.5594999999999547
0.5259999999999584


## *Try neural network*

In [87]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
import tensorflow.keras.utils as utils
from tensorflow.keras import backend

In [103]:
X_train = Xtrain/255
X_test = Xtest/255
y_train = utils.to_categorical(Ytrain)
y_test = utils.to_categorical(Ytest)
num_pixels = X_train.shape[1]
num_classes = y_test.shape[1]

In [110]:
def fully_conncected_model():
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels,kernel_initializer='normal',activation='tanh'))
    model.add(Dense(num_classes, kernel_initializer='normal',activation='softmax'))
    model.compile(loss='mean_squared_error',optimizer='sgd',metrics=['accuracy'])
    return model

model = fully_conncected_model()

In [118]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=159,batch_size=100,verbose=2)

Train on 60000 samples, validate on 10000 samples
Epoch 1/159
 - 5s - loss: 0.0209 - acc: 0.8598 - val_loss: 0.0213 - val_acc: 0.8559
Epoch 2/159
 - 4s - loss: 0.0209 - acc: 0.8598 - val_loss: 0.0213 - val_acc: 0.8556
Epoch 3/159
 - 5s - loss: 0.0209 - acc: 0.8603 - val_loss: 0.0213 - val_acc: 0.8561
Epoch 4/159
 - 4s - loss: 0.0208 - acc: 0.8602 - val_loss: 0.0213 - val_acc: 0.8555
Epoch 5/159
 - 4s - loss: 0.0208 - acc: 0.8604 - val_loss: 0.0213 - val_acc: 0.8561
Epoch 6/159
 - 4s - loss: 0.0208 - acc: 0.8606 - val_loss: 0.0213 - val_acc: 0.8561
Epoch 7/159
 - 4s - loss: 0.0208 - acc: 0.8606 - val_loss: 0.0213 - val_acc: 0.8567
Epoch 8/159
 - 5s - loss: 0.0208 - acc: 0.8607 - val_loss: 0.0212 - val_acc: 0.8565
Epoch 9/159
 - 4s - loss: 0.0208 - acc: 0.8607 - val_loss: 0.0212 - val_acc: 0.8568
Epoch 10/159
 - 5s - loss: 0.0208 - acc: 0.8610 - val_loss: 0.0212 - val_acc: 0.8566
Epoch 11/159
 - 5s - loss: 0.0207 - acc: 0.8611 - val_loss: 0.0212 - val_acc: 0.8559
Epoch 12/159
 - 5s - los

Epoch 97/159
 - 5s - loss: 0.0198 - acc: 0.8674 - val_loss: 0.0205 - val_acc: 0.8613
Epoch 98/159
 - 4s - loss: 0.0198 - acc: 0.8679 - val_loss: 0.0205 - val_acc: 0.8616
Epoch 99/159
 - 5s - loss: 0.0198 - acc: 0.8680 - val_loss: 0.0204 - val_acc: 0.8611
Epoch 100/159
 - 5s - loss: 0.0198 - acc: 0.8679 - val_loss: 0.0204 - val_acc: 0.8613
Epoch 101/159
 - 5s - loss: 0.0198 - acc: 0.8680 - val_loss: 0.0204 - val_acc: 0.8605
Epoch 102/159
 - 5s - loss: 0.0198 - acc: 0.8681 - val_loss: 0.0204 - val_acc: 0.8611
Epoch 103/159
 - 5s - loss: 0.0197 - acc: 0.8683 - val_loss: 0.0204 - val_acc: 0.8611
Epoch 104/159
 - 4s - loss: 0.0197 - acc: 0.8682 - val_loss: 0.0204 - val_acc: 0.8619
Epoch 105/159
 - 4s - loss: 0.0197 - acc: 0.8684 - val_loss: 0.0204 - val_acc: 0.8613
Epoch 106/159
 - 5s - loss: 0.0197 - acc: 0.8683 - val_loss: 0.0204 - val_acc: 0.8611
Epoch 107/159
 - 5s - loss: 0.0197 - acc: 0.8680 - val_loss: 0.0204 - val_acc: 0.8618
Epoch 108/159
 - 4s - loss: 0.0197 - acc: 0.8684 - val_lo

In [120]:
scores = model.evaluate(X_test,y_test,verbose=2)
print("Validation Accuracy: %.2f%%"%(scores[1]*100))

Validation Accuracy: 86.31%


In [121]:
scores

[0.02006090875789523, 0.8631]